In [51]:
import re
import polars as pl

In [52]:
df = pl.read_csv(r'./data/coaster_db.csv')
df.head()

coaster_name,Length,Speed,Location,Status,Opening date,Type,Manufacturer,Height restriction,Model,Height,Inversions,Lift/launch system,Cost,Trains,Park section,Duration,Capacity,G-force,Designer,Max vertical angle,Drop,Soft opening date,Fast Lane available,Replaced,Track layout,Fastrack available,Soft opening date.1,Closing date,Opened,Replaced by,Website,Flash Pass Available,Must transfer from wheelchair,Theme,Single rider line available,Restraint Style,Flash Pass available,Acceleration,Restraints,Name,year_introduced,latitude,longitude,Type_Main,opening_date_clean,speed1,speed2,speed1_value,speed1_unit,speed_mph,height_value,height_unit,height_ft,Inversions_clean,Gforce_clean
str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,f64,f64,str,str,str,str,f64,str,f64,f64,str,f64,i64,f64
"""Switchback Railway""","""600 ft (180 m)""","""6 mph (9.7 km/h)""","""Coney Island""","""Removed""","""June 16, 1884""","""Wood""","""LaMarcus Adna Thompson""",null,"""Lift Packed""","""50 ft (15 m)""",null,"""gravity""",null,null,"""Coney Island Cyclone Site""","""1:00""","""1600 riders per hour""","""2.9""","""LaMarcus Adna Thompson""","""30°""","""43 ft (13 m)""",null,null,null,"""Gravity pulled coaster""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1884,40.574,-73.978,"""Wood""","""1884-06-16""","""6 mph ""","""9.7 km/h""",6.0,"""mph""",6.0,50.0,"""ft""",null,0,2.9
"""Flip Flap Railway""",null,null,"""Sea Lion Park""","""Removed""","""1895""","""Wood""","""Lina Beecher""",null,null,null,1,null,null,"""a single car. Riders are arran…",null,null,null,"""12""","""Lina Beecher""",null,null,null,null,null,null,null,null,"""1902""",null,null,null,null,null,null,null,null,null,null,null,null,1895,40.578,-73.979,"""Wood""","""1895-01-01""",null,null,null,null,null,null,null,null,1,12.0
"""Switchback Railway (Euclid Bea…",null,null,"""Cleveland, Ohio, United States""","""Closed""",null,"""Other""",null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""1895""",null,null,null,null,null,null,null,null,null,null,null,1896,41.58,-81.57,"""Other""",null,null,null,null,null,null,null,null,null,0,null
"""Loop the Loop (Coney Island)""",null,null,"""Other""","""Removed""","""1901""","""Steel""","""Edwin Prescott""",null,null,null,1,null,null,"""a single car. Riders are arran…",null,null,null,null,"""Edward A. Green""",null,null,null,null,"""Switchback Railway""",null,null,null,"""1910""",null,"""Giant Racer""",null,null,null,null,null,null,null,null,null,null,1901,40.5745,-73.978,"""Steel""","""1901-01-01""",null,null,null,null,null,null,null,null,1,null
"""Loop the Loop (Young's Pier)""",null,null,"""Other""","""Removed""","""1901""","""Steel""","""Edwin Prescott""",null,null,null,1,null,null,null,null,null,null,null,"""Edward A. Green""",null,null,null,null,null,null,null,null,"""1912""",null,null,null,null,null,null,null,null,null,null,null,null,1901,39.3538,-74.4342,"""Steel""","""1901-01-01""",null,null,null,null,null,null,null,null,1,null


### Filter columns


In [53]:
df = df[[
    'coaster_name', 'Location', 'Status', 'Type_Main', 'Manufacturer',
    'Length', 'Height', 'speed2', 'Duration', 'Capacity', 'Gforce_clean',
    'opening_date_clean', 'Closing date', 'latitude', 'longitude'
]]
df.head(1)

coaster_name,Location,Status,Type_Main,Manufacturer,Length,Height,speed2,Duration,Capacity,Gforce_clean,opening_date_clean,Closing date,latitude,longitude
str,str,str,str,str,str,str,str,str,str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""Removed""","""Wood""","""LaMarcus Adna Thompson""","""600 ft (180 m)""","""50 ft (15 m)""","""9.7 km/h""","""1:00""","""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Rename columns


In [54]:
df = df.rename(lambda col: col.lower().replace(' ', '_'))
df = df.rename({
    'coaster_name': 'name',
    'type_main': 'material',
    'opening_date_clean': 'opening_date',
    'closing_date': 'closing_year',
    'speed2': 'speed_in_kmh',
    'duration': 'duration',
    'gforce_clean': 'g_force',
    'length': 'length_in_m',
    'height': 'height_in_m'
})
df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,str,str,str,str,str,str,str,str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""Removed""","""Wood""","""LaMarcus Adna Thompson""","""600 ft (180 m)""","""50 ft (15 m)""","""9.7 km/h""","""1:00""","""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Change status column's values


In [55]:
def change_status(status: str) -> str:
    temporary_closed = (
        'Temporarily closed', 'Temporarily Closed',
    )

    closed = (
        'Closed', 'Closed in 2021', 'Not Currently Operating',
        'Discontinued', 'Removed',
        'Chapter 7 bankruptcy; rides dismantled and sold; property sold'
    )

    sbno = (
        'SBNO (Standing But Not Operating)', 'SBNO December 2019'
    )

    under_maintenance = (
        'Under Maintenance',
        'closed for maintenance as of july 30 no reopening date known'
    )

    result = ''

    if status in temporary_closed:
        result = 'temporary_closed'
    elif status in closed:
        result = 'closed'
    elif status in sbno:
        result = 'sbno'
    elif status in under_maintenance:
        result = 'under_maintenance'
    elif status == 'Under construction':
        result = 'In Production'
    else:
        result = status

    return result.lower().replace(' ', '_')


print('Before:')
display(df['status'].unique().to_list())

df = df.with_columns(pl.col('status').fill_null('unknown'))
df = df.with_columns(pl.col('status').map_elements(
    change_status, return_dtype=pl.String).cast(pl.Categorical))

print('After:')
display(df['status'].unique().to_list())

Before:


['SBNO December 2019',
 None,
 'Temporarily closed',
 'Removed',
 'closed for maintenance as of july 30 no reopening date known',
 'Temporarily Closed',
 'Not Currently Operating',
 'Chapter 7 bankruptcy; rides dismantled and sold; property sold',
 'In Production',
 'Closed in 2021',
 'SBNO (Standing But Not Operating)',
 'Discontinued',
 'Under Maintenance',
 'Under construction',
 'Closed',
 'Operating']

After:


['closed',
 'operating',
 'unknown',
 'in_production',
 'under_maintenance',
 'sbno',
 'temporary_closed']

### Convert material column type to categorical


In [56]:
df = df.with_columns(
    pl.col('material').str.to_lowercase().cast(pl.Categorical)
)
df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,str,str,str,str,str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""","""600 ft (180 m)""","""50 ft (15 m)""","""9.7 km/h""","""1:00""","""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Fill null values in manufacturer column with unknown


In [57]:
df = df.with_columns(pl.col('manufacturer').fill_null('unknown'))
df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,str,str,str,str,str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""","""600 ft (180 m)""","""50 ft (15 m)""","""9.7 km/h""","""1:00""","""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Extract meter in length & height columns


In [58]:
def extract_meter(value: str) -> float:
    match = re.search(r"\(([\d.,]+)\s*m\)", value)

    if match:
        return float(match.group(1).replace(',', ''))
    else:
        return None


df = df.with_columns(
    pl.col("length_in_m").map_elements(extract_meter, return_dtype=pl.Float64),
    pl.col("height_in_m").map_elements(extract_meter, return_dtype=pl.Float64)
)
df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,str,str,str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,"""9.7 km/h""","""1:00""","""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Extract km/h in speed column


In [59]:
def extract_kmh(value: str) -> float:
    MPH_TO_KMH = 1.60934

    if "mp" in value:
        # Convert mph to km/h
        return float(re.search(r"[\d.]+", value).group()) * MPH_TO_KMH
    elif "km" in value:
        return float(re.search(r"[\d.]+", value).group())
    else:
        return None


df = df.with_columns(
    pl.col("speed_in_kmh").map_elements(extract_kmh, return_dtype=pl.Float64)
)
df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,f64,str,str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,9.7,"""1:00""","""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Convert duration_in_sec column values format to second


In [60]:
def extract_second(value: str) -> int:
    value = value.lower().strip()  # Normalize case and strip spaces

    # Handle number:number format (e.g., 1:48)
    match = re.match(r'^(?P<minutes>\d+):(?P<seconds>\d+)', value)
    if match:
        minutes = int(match.group('minutes'))
        seconds = int(match.group('seconds'))
        return minutes * 60 + seconds

    # Handle patterns like "1 minute 30 seconds", "1 min 30 sec"
    match = re.match(
        r'(?P<minutes>\d+)\s*min(?:ute)?s?\s*(?P<seconds>\d+)\s*sec(?:ond)?s?', value)
    if match:
        minutes = int(match.group('minutes'))
        seconds = int(match.group('seconds'))
        return minutes * 60 + seconds

    # Handle patterns like "1 minute" or "90 seconds"
    match = re.match(r'(?P<minutes>\d+)\s*min(?:ute)?s?', value)
    if match:
        return int(match.group('minutes')) * 60
    match = re.match(r'(?P<seconds>\d+)\s*sec(?:ond)?s?', value)
    if match:
        return int(match.group('seconds'))

    # Handle ranges like "between 1 minute 20 seconds and 1 minute 30 seconds"
    match = re.match(r'between\s*(?P<min_minutes>\d+)\s*min(?:ute)?s?\s*(?P<min_seconds>\d+)\s*sec(?:ond)?s?\s*and\s*(?P<max_minutes>\d+)\s*min(?:ute)?s?\s*(?P<max_seconds>\d+)\s*sec(?:ond)?s?', value)
    if match:
        min_minutes = int(match.group('min_minutes'))
        min_seconds = int(match.group('min_seconds'))
        max_minutes = int(match.group('max_minutes'))
        max_seconds = int(match.group('max_seconds'))
        min_total = min_minutes * 60 + min_seconds
        max_total = max_minutes * 60 + max_seconds
        return (min_total + max_total) // 2  # Average of the range

    # Handle approximate values and extra characters (strip non-numeric characters)
    value = re.sub(r'[^0-9:]', '', value)
    if value.isdigit():
        return int(value)

    return None  # For unhandled cases


df = df.with_columns(
    (
        pl.col('duration').map_elements(
            extract_second, return_dtype=pl.Int64
        ) * 1000
    ).cast(pl.Duration('ms'))
)
df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,f64,duration[ms],str,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,9.7,1m,"""1600 riders per hour""",2.9,"""1884-06-16""",null,40.574,-73.978


### Extract number of riders in capacity column


In [61]:
def extract_capacity(value: str) -> int:
    value = value.replace(',', '')

    return int(re.search(r"\d+", value).group())


df = df.with_columns(
    pl.col("capacity").map_elements(extract_capacity, return_dtype=pl.Int64)
)

df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,f64,duration[ms],i64,f64,str,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,9.7,1m,1600,2.9,"""1884-06-16""",null,40.574,-73.978


### Cast the type of opening_date column to pl.Date


In [62]:
df = df.with_columns(
    pl.col("opening_date").str.strptime(pl.Date, format=r"%Y-%m-%d")
)

df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,f64,duration[ms],i64,f64,date,str,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,9.7,1m,1600,2.9,1884-06-16,null,40.574,-73.978


### Extract the year & cast the type of closing_year column to pl.Int16


In [63]:
def extract_year(value: str) -> int:
    # Regex to find a 4-digit year, a decade (e.g., 1990s), or a date with a year
    match = re.search(
        r'\b(\d{4})\b|(\d{4})s\b|(\d{1,2})\s*[A-Za-z]+\s*(\d{4})', value)

    if match:
        # Return the year: if it's a decade, return the first four digits; otherwise, return the found year
        return int(match.group(1)) if match.group(1) else int(match.group(2)) if match.group(2) else int(match.group(4))
    else:
        return None  # If no match is found


df = df.with_columns(
    pl.col("closing_year").map_elements(extract_year, return_dtype=pl.Int16)
)

df.head(1)

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,f64,duration[ms],i64,f64,date,i16,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,9.7,1m,1600,2.9,1884-06-16,null,40.574,-73.978


### Drop cuplicate rows based on the combination of name & location columns


In [64]:
print(f'Before = {df.shape}')

df = df.unique(
    subset=["name", "location"],
    maintain_order=True  # True = don't sort
)

print(f'After  = {df.shape}')

Before = (1087, 15)
After  = (990, 15)


### Display cleaned data


In [65]:
df

name,location,status,material,manufacturer,length_in_m,height_in_m,speed_in_kmh,duration,capacity,g_force,opening_date,closing_year,latitude,longitude
str,str,cat,cat,str,f64,f64,f64,duration[ms],i64,f64,date,i16,f64,f64
"""Switchback Railway""","""Coney Island""","""closed""","""wood""","""LaMarcus Adna Thompson""",180.0,15.0,9.7,1m,1600,2.9,1884-06-16,null,40.574,-73.978
"""Flip Flap Railway""","""Sea Lion Park""","""closed""","""wood""","""Lina Beecher""",null,null,null,null,null,12.0,1895-01-01,1902,40.578,-73.979
"""Switchback Railway (Euclid Bea…","""Cleveland, Ohio, United States""","""closed""","""other""","""unknown""",null,null,null,null,null,null,null,null,41.58,-81.57
"""Loop the Loop (Coney Island)""","""Other""","""closed""","""steel""","""Edwin Prescott""",null,null,null,null,null,null,1901-01-01,1910,40.5745,-73.978
"""Loop the Loop (Young's Pier)""","""Other""","""closed""","""steel""","""Edwin Prescott""",null,null,null,null,null,null,1901-01-01,1912,39.3538,-74.4342
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Ice Breaker (roller coaster)""","""SeaWorld Orlando""","""in_production""","""steel""","""Premier Rides""",580.0,28.0,84.0,null,null,null,2022-02-01,null,28.4088,-81.4633
"""Leviathan (Sea World)""","""Sea World""","""in_production""","""wood""","""Martin & Vleminckx""",null,null,79.984198,null,720,null,2022-01-01,null,-27.9574,153.4263
"""Pantheon (roller coaster)""","""Busch Gardens Williamsburg""","""in_production""","""steel""","""Intamin""",1014.0,54.0,117.0,null,null,null,2022-01-01,null,37.2339,-76.6426


### Export cleaned data into a Feather file


In [66]:
df.write_ipc('./data/coaster_db-cleaned.arrow')